In [1]:
import pandas as pd
import numpy as np
import sklearn
import os
import os.path as op

In [2]:
path = '../../data/kaggle-titanic'
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
setNames = ['train_RescaleClean.csv','train_label.csv','test_RescaleClean.csv']
data = []
for sn in setNames:
    data.append( pd.read_csv(op.join(path,sn), index_col=0))
    print(data[-1].head())
    print(data[-1].info())

../data/kaggle-titanic/train_label.csv
../data/kaggle-titanic/test_RescaleClean.csv
../data/kaggle-titanic/train_RescaleClean.csv
../data/kaggle-titanic/test.csv
../data/kaggle-titanic/train.csv
../data/kaggle-titanic/significantFeatures.txt
   Pclass  Sex  SibSp  Parch  Fare_group  Embarked  Age_group  Title
0     1.0  0.0  0.125    0.0         0.0       0.0        0.2    0.0
1     0.0  1.0  0.125    0.0         0.0       0.5        0.4    0.0
2     1.0  1.0  0.000    0.0         0.0       0.0        0.2    1.0
3     0.0  1.0  0.125    0.0         0.0       0.0        0.4    0.0
4     1.0  0.0  0.000    0.0         0.0       0.0        0.4    0.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    float64
 1   Sex         891 non-null    float64
 2   SibSp       891 non-null    float64
 3   Parch       891 non-null    fl

In [3]:
# simple model
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X, y = data[0], data[1]['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
clf = MLPClassifier(hidden_layer_sizes=(20,), 
                    random_state=1, max_iter = 5000, 
                    learning_rate='adaptive', 
                    tol=1e-9
                   )
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
np.round(accuracy_score(y_test, y_pred),4)

0.7444

In [4]:
# stagged models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split

X, y = data[0], data[1]['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
estimators = [('tree', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)),
              ('mlp', MLPClassifier(hidden_layer_sizes=(100,), random_state=1, max_iter = 5000, learning_rate='adaptive', tol=1e-9)),
              ('svm', svm.SVC()),('sgd', SGDClassifier()), ('NB', GaussianNB())]

clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X, y)
y_pred = clf.predict(X_test)
np.round(accuracy_score(y_test, y_pred),4)

0.8111